# Preprocess data for machine learning

### March 3, 2018
### Tiffany Huang

In [333]:
%matplotlib inline
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV

# Input data files are available in the "WDataFiles/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "WDataFiles"]).decode("utf8"))

WCities.csv
WFieldGoalPercentage.csv
WFieldGoalPercentage_IDs.csv
WGameCities.csv
WNCAATourneyCompactResults.csv
WNCAATourneySeeds.csv
WNCAATourneySlots.csv
WReboundMargin.csv
WReboundMargin_IDs.csv
WRegularSeasonCompactResults.csv
WSampleSubmissionStage1.csv
WScoringMargin.csv
WScoringMargin_IDs.csv
WScoringOffense.csv
WScoringOffense_IDs.csv
WSeasons.csv
WTeamSpellings.csv
WTeams.csv
WTurnovers.csv
WTurnovers_IDs.csv
ranking_macro.xlsm



# Load the Training Data

In [291]:
data_dir = 'WDataFiles/'
df_seeds = pd.read_csv(data_dir + 'WNCAATourneySeeds.csv')
df_tourney = pd.read_csv(data_dir + 'WNCAATourneyCompactResults.csv')
df_reg_season = pd.read_csv(data_dir + 'WRegularSeasonCompactResults.csv')
df_field_goal = pd.read_csv(data_dir + 'WFieldGoalPercentage_IDs.csv')
df_rebounds = pd.read_csv(data_dir + 'WReboundMargin_IDs.csv')
df_scoring_offense = pd.read_csv(data_dir + 'WScoringOffense_IDs.csv')
df_turnovers = pd.read_csv(data_dir + 'WTurnovers_IDs.csv')

# Seeds Data

In [292]:
df_seeds.head()

,Season,Seed,TeamID
0,1998,W01,3330
1,1998,W02,3163
2,1998,W03,3112
3,1998,W04,3301
4,1998,W05,3272


In [293]:
# First, we'll simplify the datasets to remove the columns we won't be using 
# and convert the seedings to the needed format (stripping the regional abbreviation in front of the seed).

# Get just the digits from the seeding. Return as int
def seed_to_int(seed):
    s_int = int(seed[1:3])
    return s_int

df_seeds.drop(df_seeds[df_seeds.Season < 2002].index, inplace=True)
df_seeds.index = range(len(df_seeds))
df_seeds['seed_int'] = df_seeds.Seed.apply(seed_to_int)
df_seeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
df_seeds.head()

,Season,TeamID,seed_int
0,2002,3181,1
1,2002,3124,2
2,2002,3376,3
3,2002,3400,4
4,2002,3256,5


# Tournament Data

In [294]:
df_tourney.drop(labels=['DayNum', 'WScore', 'LScore', 'NumOT', 'WLoc'], inplace=True, axis=1)
df_tourney.drop(df_tourney[df_tourney.Season < 2002].index, inplace=True)
df_tourney.index = range(len(df_tourney))
df_tourney.head()

,Season,WTeamID,LTeamID
0,2002,3116,3155
1,2002,3124,3137
2,2002,3160,3380
3,2002,3179,3393
4,2002,3181,3313


In [295]:
# Merge the Seeds with their corresponding TeamIDs in the compact results dataframe.
df_winseeds = df_seeds.rename(columns={'TeamID':'WTeamID', 'seed_int':'WSeed'})
df_lossseeds = df_seeds.rename(columns={'TeamID':'LTeamID', 'seed_int':'LSeed'})
df_dummy = pd.merge(left=df_tourney, right=df_winseeds, how='left', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_dummy, right=df_lossseeds, on=['Season', 'LTeamID'])
df_concat['SeedDiff'] = df_concat.WSeed - df_concat.LSeed
df_concat.drop(labels=['WSeed', 'LSeed'], inplace=True, axis=1)
df_concat.head()

,Season,WTeamID,LTeamID,SeedDiff
0,2002,3116,3155,-5
1,2002,3124,3137,-13
2,2002,3160,3380,-11
3,2002,3179,3393,-3
4,2002,3181,3313,-15


# Regular Season Data

# Field Goal Percentage Data

In [296]:
df_field_goal.drop(labels=['Team', 'Kaggle Name', 'Ranking'], inplace=True, axis=1)
df_field_goal.drop(df_field_goal[df_field_goal.Season > 2017].index, inplace=True)
df_field_goal['FG Percentage'] = df_field_goal['FG Percentage'] / 100
df_field_goal['Kaggle TeamID'] = df_field_goal['Kaggle TeamID'].astype(int)
df_field_goal.head()

,Season,FG Percentage,Kaggle TeamID
0,2002,0.521,3435
1,2002,0.520,3163
2,2002,0.494,3243
3,2002,0.493,3278
4,2002,0.490,3181


In [297]:
df_win_fg = df_field_goal.rename(columns={'Kaggle TeamID':'WTeamID', 'FG Percentage':'WFGP'})
df_lose_fg = df_field_goal.rename(columns={'Kaggle TeamID':'LTeamID', 'FG Percentage':'LFGP'})
df_concat = pd.merge(left=df_concat, right=df_win_fg, how='inner', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_lose_fg, how='inner', on=['Season', 'LTeamID'])
df_concat['FG%Diff'] = df_concat.WFGP - df_concat.LFGP
df_concat.drop(labels=['WFGP', 'LFGP'], inplace=True, axis=1)

# Rebound Margin Data

In [298]:
df_rebounds.drop(labels=['Team', 'Kaggle Name', 'Ranking', 'Rebound Margin'], inplace=True, axis=1)
df_rebounds.drop(df_rebounds[df_rebounds.Season > 2017].index, inplace=True)
df_rebounds['Kaggle TeamID'] = df_rebounds['Kaggle TeamID'].astype(int)
df_rebounds.head()

,Season,RPG,Kaggle TeamID
0,2002,45.5,3163
1,2002,48.6,3256
2,2002,42.8,3330
3,2002,43.7,3198
4,2002,44.6,3412


In [299]:
df_win_rpg = df_rebounds.rename(columns={'Kaggle TeamID':'WTeamID', 'RPG':'WRPG'})
df_lose_rpg = df_rebounds.rename(columns={'Kaggle TeamID':'LTeamID', 'RPG':'LRPG'})
df_concat = pd.merge(left=df_concat, right=df_win_rpg, how='inner', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_lose_rpg, how='inner', on=['Season', 'LTeamID'])
df_concat['ReboundDiff'] = df_concat.WRPG - df_concat.LRPG
df_concat.drop(labels=['WRPG', 'LRPG'], inplace=True, axis=1)
df_concat.head()

,Season,WTeamID,LTeamID,SeedDiff,FG%Diff,ReboundDiff
0,2002,3116,3155,-5,0.017,-1.3
1,2002,3124,3137,-13,0.006,5.5
2,2002,3160,3380,-11,0.044,-0.8
3,2002,3160,3261,-3,-0.005,2.6
4,2002,3160,3390,1,-0.021,-4.7


# Scoring Data

In [300]:
df_scoring_offense.drop(labels=['Team', 'Kaggle Name', 'Rank','Losses','Total Points'], inplace=True, axis=1)
df_scoring_offense.drop(df_field_goal[df_scoring_offense.Season > 2017].index, inplace=True)
df_scoring_offense['Win%'] = df_scoring_offense['Wins'] / df_scoring_offense['Games Played']
df_scoring_offense['Kaggle TeamID'] = df_scoring_offense['Kaggle TeamID'].astype(int)
df_scoring_offense.head()

/Users/tiffany/miniconda3/envs/carnd-term1/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


,Season,Games Played,Wins,PPG,Kaggle TeamID,Win%
0,2002,39,39,87.0,3163,1.000000
1,2002,35,31,83.5,3181,0.885714
2,2002,31,23,80.9,3184,0.741935
3,2002,35,32,80.3,3390,0.914286
4,2002,35,26,79.9,3314,0.742857


In [301]:
df_win_ppg = df_scoring_offense.rename(columns={'Kaggle TeamID':'WTeamID', 'PPG':'WPPG'})
df_win_ppg.drop(labels=['Games Played','Wins','Win%'], inplace=True, axis=1)
df_lose_ppg = df_scoring_offense.rename(columns={'Kaggle TeamID':'LTeamID', 'PPG':'LPPG'})
df_lose_ppg.drop(labels=['Games Played','Wins','Win%'], inplace=True, axis=1)
df_concat = pd.merge(left=df_concat, right=df_win_ppg, how='inner', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_lose_ppg, how='inner', on=['Season', 'LTeamID'])
df_concat['PPGDiff'] = df_concat.WPPG - df_concat.LPPG
df_concat.drop(labels=['WPPG', 'LPPG'], inplace=True, axis=1)
df_concat.head()

,Season,WTeamID,LTeamID,SeedDiff,FG%Diff,ReboundDiff,PPGDiff
0,2002,3116,3155,-5,0.017,-1.3,5.6
1,2002,3124,3137,-13,0.006,5.5,5.7
2,2002,3160,3380,-11,0.044,-0.8,7.8
3,2002,3160,3261,-3,-0.005,2.6,3.2
4,2002,3160,3390,1,-0.021,-4.7,-4.2


In [302]:
df_win_win_percent = df_scoring_offense.rename(columns={'Kaggle TeamID':'WTeamID', 'Win%':'WWin%'})
df_win_win_percent.drop(labels=['Games Played', 'Wins', 'PPG'], inplace=True, axis=1)
df_lose_win_percent = df_scoring_offense.rename(columns={'Kaggle TeamID':'LTeamID', 'Win%':'LWin%'})
df_lose_win_percent.drop(labels=['Games Played', 'Wins', 'PPG'], inplace=True, axis=1)
df_concat = pd.merge(left=df_concat, right=df_win_win_percent, how='inner', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_lose_win_percent, how='inner', on=['Season', 'LTeamID'])
df_concat['Win%Diff'] = df_concat['WWin%'] - df_concat['LWin%']
df_concat.drop(labels=['WWin%', 'LWin%'], inplace=True, axis=1)
df_concat.head()

,Season,WTeamID,LTeamID,SeedDiff,FG%Diff,ReboundDiff,PPGDiff,Win%Diff
0,2002,3116,3155,-5,0.017,-1.3,5.6,0.038793
1,2002,3124,3137,-13,0.006,5.5,5.7,0.140762
2,2002,3160,3380,-11,0.044,-0.8,7.8,-0.132827
3,2002,3160,3261,-3,-0.005,2.6,3.2,0.105882
4,2002,3160,3390,1,-0.021,-4.7,-4.2,-0.208403


# Turnover Data

In [303]:
df_turnovers.drop(labels=['Team', 'Kaggle Name', 'Ranking'], inplace=True, axis=1)
df_turnovers.drop(df_turnovers[df_turnovers.Season > 2017].index, inplace=True)
df_turnovers['Kaggle TeamID'] = df_turnovers['Kaggle TeamID'].astype(int)
df_turnovers.head()

,Season,TOPG,Kaggle TeamID
0,2002,11.3,3437
1,2002,12.6,3375
2,2002,13.5,3261
3,2002,13.5,3153
4,2002,13.5,3116


In [304]:
df_win_turnovers = df_turnovers.rename(columns={'Kaggle TeamID':'WTeamID', 'TOPG':'WTOPG'})
df_lose_turnovers = df_turnovers.rename(columns={'Kaggle TeamID':'LTeamID', 'TOPG':'LTOPG'})
df_concat = pd.merge(left=df_concat, right=df_win_turnovers, how='inner', on=['Season', 'WTeamID'])
df_concat = pd.merge(left=df_concat, right=df_lose_turnovers, how='inner', on=['Season', 'LTeamID'])
df_concat['TOPGDiff'] = df_concat.WTOPG - df_concat.LTOPG
df_concat.drop(labels=['WTOPG', 'LTOPG'], inplace=True, axis=1)
df_concat.head()

,Season,WTeamID,LTeamID,SeedDiff,FG%Diff,ReboundDiff,PPGDiff,Win%Diff,TOPGDiff
0,2002,3116,3155,-5,0.017,-1.3,5.6,0.038793,-1.6
1,2002,3124,3137,-13,0.006,5.5,5.7,0.140762,-2.0
2,2002,3160,3380,-11,0.044,-0.8,7.8,-0.132827,-2.2
3,2002,3160,3261,-3,-0.005,2.6,3.2,0.105882,0.9
4,2002,3160,3390,1,-0.021,-4.7,-4.2,-0.208403,-2.2


In [305]:
# Now we'll create a dataframe that summarizes wins & losses along with 
# their corresponding seed differences. This is the meat of what we'll be creating our model on.
df_wins = pd.DataFrame()
df_wins = df_concat.drop(labels=['Season', 'WTeamID', 'LTeamID'], axis=1)
df_wins['Result'] = 1

df_losses = pd.DataFrame()
df_losses['SeedDiff'] = -df_concat['SeedDiff']
df_losses['FG%Diff'] = -df_concat['FG%Diff']
df_losses['ReboundDiff'] = -df_concat['ReboundDiff']
df_losses['PPGDiff'] = -df_concat['PPGDiff']
df_losses['Win%Diff'] = -df_concat['Win%Diff']
df_losses['TOPGDiff'] = -df_concat['TOPGDiff']
df_losses['Result'] = 0

df_predictions = pd.concat((df_wins, df_losses))
df_predictions = (df_predictions - df_predictions.min()) / (df_predictions.max() - df_predictions.min())
df_predictions

,SeedDiff,FG%Diff,ReboundDiff,PPGDiff,Win%Diff,TOPGDiff,Result
0,0.333333,0.553797,0.452206,0.591803,0.526367,0.430435,1.0
1,0.066667,0.518987,0.702206,0.593443,0.595673,0.413043,1.0
2,0.133333,0.639241,0.470588,0.627869,0.409720,0.404348,1.0
3,0.400000,0.484177,0.595588,0.552459,0.571966,0.539130,1.0
4,0.533333,0.433544,0.327206,0.431148,0.358353,0.404348,1.0
5,0.400000,0.610759,0.507353,0.575410,0.620256,0.478261,1.0
6,0.666667,0.518987,0.275735,0.434426,0.458807,0.447826,1.0
7,0.000000,0.867089,0.419118,0.757377,0.619648,0.395652,1.0
8,0.266667,0.731013,0.437500,0.700000,0.575798,0.386957,1.0
9,0.400000,0.705696,0.477941,0.713115,0.634722,0.426087,1.0


In [306]:
X_train = df_predictions[['SeedDiff','FG%Diff','ReboundDiff','PPGDiff','Win%Diff','TOPGDiff']].values.reshape(-1,6)
y_train = df_predictions.Result.values
X_train, y_train = shuffle(X_train, y_train)

# Train the Model

In [325]:
# Use a basic logistic regression to train the model. You can set different C values to see how performance changes.
logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=3, num=9)}
clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

Best log_loss: -0.448, with best C: 100.0


forest = RandomForestClassifier()
params = {'max_depth': np.logspace(start=2, stop=10, num=13)}
clf = GridSearchCV(forest, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)
print('Best log_loss: {:.4}, with best depth: {}'.format(clf.best_score_, clf.best_params_['max_depth']))

In [334]:
boosting = GradientBoostingClassifier()
params = {'learning_rate': np.logspace(start=0.1, stop=1, num=10)}
clf = GridSearchCV(boosting, params, scoring='neg_log_loss', refit=True)
clf.fit(X_train, y_train)

print('Best log_loss: {:.4}, with best learning rate: {}'.format(clf.best_score_, clf.best_params_['learning_rate']))

Best log_loss: -0.1227, with best learning rate: 1.2589254117941673


In [335]:
df_sample_sub = pd.read_csv(data_dir + 'WSampleSubmissionStage1.csv')
n_test_games = len(df_sample_sub)
def get_year_t1_t2(ID):
    """Return a tuple with ints `year`, `team1` and `team2`."""
    return (int(x) for x in ID.split('_'))

In [336]:
from sklearn.preprocessing import MinMaxScaler

X_test = np.zeros(shape=(n_test_games, 6))
for ii, row in df_sample_sub.iterrows():
    year, t1, t2 = get_year_t1_t2(row.ID)
    t1_seed = df_seeds[(df_seeds.TeamID == t1) & (df_seeds.Season == year)].seed_int.values[0]
    t2_seed = df_seeds[(df_seeds.TeamID == t2) & (df_seeds.Season == year)].seed_int.values[0]
    diff_seed = t1_seed - t2_seed
    X_test[ii, 0] = diff_seed
    
    t1_fgp = df_field_goal[(df_field_goal['Kaggle TeamID'] == t1) & (df_field_goal.Season == year)]['FG Percentage'].values[0]
    t2_fgp = df_field_goal[(df_field_goal['Kaggle TeamID'] == t2) & (df_field_goal.Season == year)]['FG Percentage'].values[0]
    diff_fgp = t1_fgp - t2_fgp
    X_test[ii, 1] = diff_fgp
    
    t1_rpg = df_rebounds[(df_rebounds['Kaggle TeamID'] == t1) & (df_rebounds.Season == year)]['RPG'].values[0]
    t2_rpg = df_rebounds[(df_rebounds['Kaggle TeamID'] == t2) & (df_rebounds.Season == year)]['RPG'].values[0]
    diff_rpg = t1_rpg - t2_rpg
    X_test[ii, 2] = diff_rpg
    
    t1_ppg = df_scoring_offense[(df_scoring_offense['Kaggle TeamID'] == t1) & 
                                (df_scoring_offense.Season == year)]['PPG'].values[0]
    t2_ppg = df_scoring_offense[(df_scoring_offense['Kaggle TeamID'] == t2) & 
                                (df_scoring_offense.Season == year)]['PPG'].values[0]
    diff_ppg = t1_ppg - t2_ppg
    X_test[ii, 3] = diff_ppg
    
    t1_win_percent = df_scoring_offense[(df_scoring_offense['Kaggle TeamID'] == t1) & (df_scoring_offense.Season == year)]['Win%'].values[0]
    t2_win_percent = df_scoring_offense[(df_scoring_offense['Kaggle TeamID'] == t2) & (df_scoring_offense.Season == year)]['Win%'].values[0]
    diff_win_percent = t1_win_percent - t2_win_percent
    X_test[ii, 4] = diff_win_percent
    
    t1_topg = df_turnovers[(df_turnovers['Kaggle TeamID'] == t1) & (df_turnovers.Season == year)]['TOPG'].values[0]
    t2_topg = df_turnovers[(df_turnovers['Kaggle TeamID'] == t2) & (df_turnovers.Season == year)]['TOPG'].values[0]
    diff_topg = t1_topg - t2_topg
    X_test[ii, 5] = diff_topg

scaler = MinMaxScaler()
scaler.fit(X_test)
X_test_normalized = scaler.transform(X_test)

In [324]:
preds = clf.predict_proba(X_test_normalized)[:,1]

clipped_preds = np.clip(preds, 0.05, 0.95)
df_sample_sub.Pred = clipped_preds
df_sample_sub

,ID,Pred
0,2014_3103_3107,0.496962
1,2014_3103_3113,0.487266
2,2014_3103_3119,0.503814
3,2014_3103_3124,0.312532
4,2014_3103_3140,0.494152
5,2014_3103_3143,0.494373
6,2014_3103_3151,0.491940
7,2014_3103_3163,0.324235
8,2014_3103_3169,0.507020
9,2014_3103_3173,0.482966


In [319]:
# Create submission file!
df_sample_sub.to_csv('forest_all_data.csv', index=False)